# **Preparación del notebook**

In [1]:
from google.colab import drive
import sys
import os
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
#Conectar al drive local
drive.mount('/content/drive')
#Conectar al repositorio
path = '/content/drive/My Drive/cod/CA_inventarios'
sys.path.append(path) #para importar las funciones propias a través de import, porque incluye la carpeta del repositorio como uno de esos paquetes para que import busque funciones
os.chdir(path) #para que suba y descargue archivos de la ruta del repositorio de trabajo

Mounted at /content/drive


In [3]:
#Crear los df's de cada hoja del libro de excel
hojas_excel = pd.read_excel('/content/drive/My Drive/cod/CA_inventarios/data/bd_caso.xlsx', sheet_name=["Inventario inicial", "Salidas", "Entradas", "Maestro de materiales"])
df_inventario_inicial = hojas_excel["Inventario inicial"]
df_salidas = hojas_excel["Salidas"]
df_entradas = hojas_excel["Entradas"]
df_maestro_materiales = hojas_excel["Maestro de materiales"]

In [4]:
#Conectarse al colab de funciones
%run a_funciones.ipynb

# **Exploración y tratamiento**

## Visualización de los df's

In [5]:
df_inventario_inicial.head()

,SKU,Inventario Inicial (Cajas)
0,13569,3
1,13970,574
2,13980,2380
3,17070,74
4,17090,150


In [6]:
df_salidas.head()

,Transporte,Pedido,Entrega,Destinat.,Material,CANT,Dia,Hora,Cajas por Pallet
0,30113428,13515,92389141,13515,18375,100.0,2012-11-26,07:00:00,33
1,30113428,13515,92389141,13515,20176,5.0,2012-11-26,07:00:00,25
2,30113428,13515,92389141,13515,21250,6.0,2012-11-26,07:00:00,18
3,30113428,13515,92389141,13515,21261,60.0,2012-11-26,07:00:00,28
4,30113428,13515,92389141,13515,31002,2.0,2012-11-26,07:00:00,80


In [7]:
df_entradas.head()

,Material,Cajas,Documento de recibo,Fecha,Hora
0,20176,30.0,41239,2012-11-21,09:00:00
1,34259,6.0,41239,2012-11-21,09:00:00
2,34377,1.0,41239,2012-11-21,09:00:00
3,43564,3.0,41239,2012-11-21,09:00:00
4,71101,266.0,41239,2012-11-21,09:00:00


In [8]:
df_maestro_materiales.head()

,Material,Cajas por Pallet,Categoría,m3 por caja,Valor de la mercancía por caja,Kg por caja,Tamaño/Dimensión
0,13569,55,CARNES Y CARNES FRIAS,"0,047 M3",194444,"7,050 KG",600X400X196
1,13970,44,BEBIDAS NO ALCOHOLICAS,"0,049 M3",128660,"6,090 KG",500X500X196
2,13980,66,"GALLETERIA, CONFITERIA Y CHOCOLATES","0,039 M3",151408,"4,560 KG",510X390X196
3,17070,240,PANADERIA INDUSTRIAL,"0,008 M3",298914,"2,120 KG",295X195X140
4,17090,180,COSMETICOS,"0,011 M3",161853,"2,850 KG",350X190X140


## Cambio de nombre en los df's para que en todos los productos se llamen SKU

In [9]:
df_maestro_materiales.rename(columns={'Material': 'SKU'}, inplace=True)
df_salidas.rename(columns={'Material': 'SKU'}, inplace=True)
df_entradas.rename(columns={'Material': 'SKU'}, inplace=True)

## Validación de filas duplicados, datos nulos, dastos '-' y otros errores

In [10]:
for df, nombre in [(df_inventario_inicial, 'Inventario inicial'),(df_salidas, 'Salidas'),
                   (df_entradas, 'Entradas'), (df_maestro_materiales, 'Maestro de materiales')]:

    print(f'\n{"="*40}')
    print(f'Análisis del DataFrame: {nombre}')
    print(f'\n{"="*40}')

    print('\n📝 Tipos de datos:')
    print(df.dtypes)

    print('\n❌ Valores nulos por columna:')
    print(df.isnull().sum())

    print('\n Cantidad de filas duplicadas:')
    print(df.duplicated().sum())

    print('\n Filas con valores no numéricos (como "-", ".", vacíos):')
    for col in df.columns:
        # Revisar filas con valores "-", ".", o vacíos
        invalid_values = df[col].isin(['-', '.', '']).sum()
        if invalid_values >0:
            print(f'  ⚠️ Columna "{col}" tiene {invalid_values} filas con valores no válidos (como "-", ".", vacíos)')
        else:
            print(f'  ✅ Columna "{col}" = 0')



Análisis del DataFrame: Inventario inicial


📝 Tipos de datos:
SKU                           int64
Inventario Inicial (Cajas)    int64
dtype: object

❌ Valores nulos por columna:
SKU                           0
Inventario Inicial (Cajas)    0
dtype: int64

 Cantidad de filas duplicadas:
0

 Filas con valores no numéricos (como "-", ".", vacíos):
  ✅ Columna "SKU" = 0
  ✅ Columna "Inventario Inicial (Cajas)" = 0

Análisis del DataFrame: Salidas


📝 Tipos de datos:
Transporte                   int64
Pedido                       int64
Entrega                      int64
Destinat.                    int64
SKU                          int64
CANT                       float64
Dia                 datetime64[ns]
Hora                        object
Cajas por Pallet             int64
dtype: object

❌ Valores nulos por columna:
Transporte          0
Pedido              0
Entrega             0
Destinat.           0
SKU                 0
CANT                0
Dia                 0
Hora              

# **Solución de preguntas**



##  ¿Cuántas referencias se manejan?


In [11]:
# Usamos inventrio inicial o maestro de materiales para revisar cuantos valores unicos hay en SKU
df_inventario_inicial['SKU'].nunique()

464

In [12]:
df_maestro_materiales['SKU'].nunique()

464

## ¿Cuántas referncias se reciben y se despachan?

In [13]:
len(df_salidas['SKU'].unique()) #revisar cuantos valores unicos de SKU hay en salidas

464

In [14]:
len(df_entradas['SKU'].unique()) #revisar cuantos valores unicos de SKU hay en entradas

464

## ABC de productos por rotación (según CANT)

Se suman todas las CANT que corresponden a un mismo SKU para encontrar la cantidad total de salidas de ese SKU y se divide por la suma de toda la columna de CANT que es el total de todas las salidas y eso es el porcentaje de participación de ese SKU.

In [19]:
#Se suma el total de salidas
df_salidas['CANT'] = df_salidas['CANT'].astype(int)
total_cajas = df_salidas['CANT'].sum()
print(total_cajas)

2307957


In [20]:
#Se debe agrupar todos los CANT de cada SKU y hacer la suma y frecuencia acumulada
df_rotacion = df_salidas[['SKU', 'CANT']].groupby('SKU', as_index=False)['CANT'].sum().sort_values(by='CANT', ascending=False).copy()
df_rotacion['Suma Acumulada'] = df_rotacion['CANT'].cumsum() #Crear fila de suma acumulada
df_rotacion['Porcentaje Acumulado'] = (df_rotacion['Suma Acumulada'] / total_cajas) * 100 #Crear fila de porcentaje acumulado

In [ ]:
# Filtrar los SKUs que conforman el 80% del inventario total para la clase A
rotacion_A = df_rotacion[df_rotacion['Porcentaje Acumulado'] <= 80]
# Mostrar los resultados
print(f'✅ Número de SKUs que representan el 80% del inventario y que son tipo A: {len(rotacion_A)}')
print('\n SKUs que conforman el 80%:')
print(rotacion_A['SKU'])

In [ ]:
# Filtrar los SKUs que conforman el 80%-95% del inventario para la clase B
rotacion_B = df_rotacion[(df_rotacion['Porcentaje Acumulado'] > 80) & (df_rotacion['Porcentaje Acumulado'] <= 95)]
print(f'✅ Número de SKUs que representan el 80%-95% del inventario y que son tipo B: {len(rotacion_B)}')
print('\n SKUs que conforman el 80%-95%:')
print(rotacion_B['SKU'])

In [ ]:
# Filtrar los SKUs que conforman el 95%-100% del inventario para la clase C
rotacion_C = df_rotacion[(df_rotacion['Porcentaje Acumulado'] > 95) & (df_rotacion['Porcentaje Acumulado'] <= 100)]
print(f'\n✅ Número de SKUs que representan el 95%-100% del inventario y que son tipo C: {len(rotacion_C)}')
print('\n SKUs que conforman el 95%-100%:')
print(rotacion_C['SKU'])

## ABC de popularidad (segun Pedido)

In [27]:
#Calcular cuantos pedidos unicos estan asociados a cada SKU y ordenarlos de mayor a menor
df_salidas['SKU'] = df_salidas['SKU'].astype(str)
df_popularidad = df_salidas.groupby('SKU')['Pedido'].nunique().reset_index()
df_popularidad.rename(columns={'Pedido': 'Pedidos'}, inplace=True)
df_popularidad = df_popularidad.sort_values(by='Pedidos', ascending=False).copy()

In [29]:
total_pedidos = df_popularidad['Pedidos'].sum()

In [30]:
#Calcular la suma acumulada y el porcentaje acumulado
df_popularidad['Suma Acumulada'] = df_popularidad['Pedidos'].cumsum()
df_popularidad['Porcentaje Acumulado'] = (df_popularidad['Suma Acumulada'] / total_pedidos) * 100
df_popularidad

,SKU,Pedidos,Suma Acumulada,Porcentaje Acumulado
137,40284,101,101,0.905830
291,71152,100,201,1.802691
12,18233,100,301,2.699552
8,18128,100,401,3.596413
163,43416,99,500,4.484305
...,...,...,...,...
423,97800,1,11146,99.964126
426,97807,1,11147,99.973094
427,97830,1,11148,99.982063
19,18371,1,11149,99.991031


In [31]:
# Filtrar los SKUs que conforman el 80% para la clase A
popularidad_A = df_popularidad[df_popularidad['Porcentaje Acumulado'] <= 80]
# Mostrar los resultados
print(f'✅ Número de SKUs que representan el 80% del inventario y que son tipo A: {len(popularidad_A)}')
print('\n SKUs que conforman el 80%:')
print(popularidad_A['SKU'])

✅ Número de SKUs que representan el 80% del inventario y que son tipo A: 149

 SKUs que conforman el 80%:
137    40284
291    71152
12     18233
8      18128
163    43416
       ...  
165    43514
145    41577
223    47060
288    71101
287    71100
Name: SKU, Length: 149, dtype: object


In [ ]:
# Filtrar los SKUs que conforman el 80%-95% para la clase B
popularidad_B = df_popularidad[(df_popularidad['Porcentaje Acumulado'] > 80) & (df_popularidad['Porcentaje Acumulado'] <= 95)]
print(f'✅ Número de SKUs que representan el 80%-95% del inventario y que son tipo B: {len(popularidad_B)}')
print('\n SKUs que conforman el 80%-95%:')
print(popularidad_B['SKU'])

In [ ]:
# Filtrar los SKUs que conforman el 95%-100% para la clase C
popularidad_C = df_popularidad[(df_popularidad['Porcentaje Acumulado'] > 95) & (df_popularidad['Porcentaje Acumulado'] <= 100)]
print(f'\n✅ Número de SKUs que representan el 95%-100% del inventario y que son tipo C: {len(popularidad_C)}')
print('\n SKUs que conforman el 95%-100%:')
print(popularidad_C['SKU'])